In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from flame_net.FourierOp_Nd import FourierOp_Nd
from flame_net.DeepONet_1d import DeepONet_1d
from flame_net.ConvPDE_Nd import ConvPDE_Nd
from flame_net.lib_uti import Cdata_sys, count_learnable_params, lib_Model,lib_DataGen,LpLoss,lib_ModelTrain
from timeit import default_timer

#-----------------------
device = torch.device('cuda') # if torch.cuda.is_available() else 'cpu')


#-----------------------
nDIM = 1
data_sys = Cdata_sys('siva',[0.02],1)
#data_sys = Cdata_sys('cfd',['L1536_rho8'])
params = lib_Model.set_default_params(data_sys,nDIM)

params['T_in' ] = 1
params['T_out'] = 20
params['data_channel'] = 1
#params['data:yB_estimate']=np.array([-0.7, 1.3]) * np.pi
#params['data:yB_estimate']=np.array([-0.8, 2.4]) * np.pi
#params['data:yB_estimate']=np.array([-0.7, 2.5]) * np.pi
#params['data:AspectRatio_set'] = 1 # (2.5+0.7)/2.
params['data:nStep']  = 1
params['data:nStepSkip']=1
params['Nx']            = 512 # 128
#-----
params['train:batch_size'] = 1000
params['train:learning_rate'] = 0.0025
#-----------
#model_name = 'fourier'
#params['fourier:modes_fourier' ] =  128 #64 #,64]
#params['fourier:width' ] =  30 # 20
#params['fourier:depth' ] = 4
#-------
model_name = 'conv'
params['conv:method_types_conv'] = 'inception_less'
params['conv:en1_channels' ] = [ [16],[32,32],[64,64],[128],[128],[64],[32]] 
#params['conv:en1_channels' ] = [16,32,64,64,128,64,32,16]


#params['conv:en1_channels' ] = [16,32,64,64,128,128,128,128]
#params['conv:method_skip' ]= 'width2'
#params['conv:method_nonlinear' ]= 'de'
#-----
#model_name = 'onet'
#params['onet:type_branch']='conv'
#params['onet:type_branch']='fc'
##params['onet:P'] = 50
#params['onet:trunk_featurepair']=2
#params['onet:type_trunk']='simple'
#---------

train_disp, test_disp, train_PDEpara, test_PDEpara = lib_DataGen.DataGen(data_sys,params)


----- params for DataGen -----
nDIM,T_in,T_out,Nx,nStep,nStepSkip,data_channel, data_sys.sysname, data_sys.list_nu, data_sys.list_cfdfilename
------------------------------
N= 512 ,dt_Output= 0.015 ,nu_list = [0.02] ,nu_extention_list = [0.02] ,NumOutPut_dt= [37]
ActiveNoise_Amplitude=0.000000,stepfeq=1,k = 0 
[(1, 200001, 0.02, 'rand_FFT_2_8'), (500, 201, 0.02, 'rand_FFT_2_8'), (500, 201, 0.02, 'rand_simple')]
num_traj= 1  leq_seq= 200001  nu= 0.02   rand_FFT_2_8
Sucess: load ./data/dsol_multraj1L200001_nu02_N512_dt015_rand_FFT_2_8.pkl
num_traj= 500  leq_seq= 201  nu= 0.02   rand_FFT_2_8
Sucess: load ./data/dsol_multraj500L201_nu02_N512_dt015_rand_FFT_2_8.pkl
num_traj= 500  leq_seq= 201  nu= 0.02   rand_simple
Sucess: load ./data/dsol_multraj500L201_nu02_N512_dt015_rand_simple.pkl
[(1, 20001, 0.02, 'rand_FFT_2_8'), (50, 201, 0.02, 'rand_FFT_2_8'), (50, 201, 0.02, 'rand_simple')]
num_traj= 1  leq_seq= 20001  nu= 0.02   rand_FFT_2_8
Sucess: load ./data/dsol_multraj1L20001_nu02_N512_dt01

In [2]:
#%matplotlib inline
#plt.figure(figsize=[4,4])
#plt.contourf(  train_disp[-1,:,:,-1]  )

In [3]:
model_name_detail = lib_Model.get_model_name_detail(model_name,data_sys,params)
model = lib_Model.build_model(model_name_detail,params)

Conv_nu02_inception_less_o20
ConvPDE_Nd: en_channels =  [[1, 16], [16, 32, 32], [32, 64, 64], [64, 128], [128, 128], [128, 64], [64, 32]]
ConvPDE_Nd: de_channels =  [[32, 32], [64, 64], [128, 128], [256, 128], [256, 64], [128, 32], [64, 16], [32, 1]]
ConvPDE_Nd: en_types =  [['c'], ['c', 'i'], ['c', 'i'], ['i'], ['c'], ['c'], ['c']]
ConvPDE_Nd: de_types =  [['c'], ['c'], ['c'], ['c'], ['c'], ['c'], ['i'], ['c']]
count_learnable_params= 339733


In [4]:
params['train:batch_size'] =700
lib_ModelTrain.Train(train_disp, test_disp,train_PDEpara,test_PDEpara,model,model_name_detail,device,params )


batch_size= 700
ntrain= 17998  ,ntest= 1798
train_l2_indentity= tensor(0.2097) ,  test_l2_indentity= tensor(0.2099)


/apps/Alvis/software/PyTorch/1.9.0-fosscuda-2020b/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1153.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


..........................
ep t[s] trainErr_norm testErr_norm train_l2 test_l2 train_l2_step test_l2_step train_rel_ind test_rel_ind 
0, 125.02, 1.000, 0.953, 1.018, 0.970, 1.032, 0.966, 4.852, 4.620
..........................
1, 123.87, 0.915, 0.850, 0.931, 0.865, 0.922, 0.848, 4.440, 4.120
..........................
2, 123.96, 0.817, 0.774, 0.831, 0.788, 0.823, 0.790, 3.964, 3.752
..........................
3, 123.89, 0.738, 0.699, 0.751, 0.712, 0.736, 0.697, 3.580, 3.391
..........................
4, 123.91, 0.629, 0.572, 0.640, 0.583, 0.624, 0.562, 3.052, 2.775
..........................
5, 123.87, 0.510, 0.438, 0.519, 0.446, 0.498, 0.418, 2.477, 2.124
..........................
6, 123.86, 0.410, 0.324, 0.417, 0.330, 0.391, 0.304, 1.990, 1.571
....

KeyboardInterrupt: 